*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     |████████████████████████████████| 510.6 MB 5.3 kB/s 
     |████████████████████████████████| 1.6 MB 52.6 MB/s 
     |████████████████████████████████| 438 kB 55.4 MB/s 
     |████████████████████████████████| 5.8 MB 50.7 MB/s 
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, but you have tf-estimator-nightly 2.10.0.dev2022040408 which is incompatible.
2.10.0-dev20220404


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-04-05 05:01:25--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  1.98MB/s    in 0.2s    

2022-04-05 05:01:26 (1.98 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-04-05 05:01:26--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

+ Reading tsv

In [ ]:
train_file = pd.read_csv('/content/train-data.tsv', sep='\t')
valid_file = pd.read_csv('/content/valid-data.tsv', sep='\t')

In [ ]:
train_file.head()

,ham,"ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it."
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...


+ Mapping status to integer

In [ ]:
x = train_file.copy()
y = valid_file.copy()
x.columns=['status', 'msg']
y.columns=['status', 'msg']
x["status"] = x["status"].apply(lambda xt : 0 if xt == 'ham' else 1)
y["status"] = y["status"].apply(lambda yt : 0 if yt == 'ham' else 1)

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4178 entries, 0 to 4177
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   status  4178 non-null   int64 
 1   msg     4178 non-null   object
dtypes: int64(1), object(1)
memory usage: 65.4+ KB


In [ ]:
x.head()

,status,msg
0,0,you can never do nothing
1,0,"now u sound like manky scouse boy steve,like! ..."
2,0,mum say we wan to go then go... then she can s...
3,0,never y lei... i v lazy... got wat? dat day ü ...
4,0,in xam hall boy asked girl tell me the startin...


+ Defining encoder

In [ ]:
encoder = keras.layers.TextVectorization()
encoder.adapt(x['msg'].tolist())
vocab_size = len(encoder.get_vocabulary())
print(vocab_size)

8178


+ Model

In [ ]:
model = keras.Sequential([
    encoder,
    keras.layers.Embedding(input_dim = vocab_size, output_dim = 128, mask_zero = True),
    keras.layers.Bidirectional(keras.layers.LSTM(128)),
    keras.layers.Dense(16, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 128)         1046784   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              263168    
 l)                                                              
                                                                 
 dense (Dense)               (None, 16)                4112      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,314,081
Trainable params: 1,314,081
Non-

In [ ]:
model.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [ ]:
history = model.fit(x = x['msg'], y = x['status'], epochs = 5, validation_data = (y['msg'], y['status']))

Epoch 1/5
131/131 [==============================] - 37s 231ms/step - loss: 0.1832 - accuracy: 0.9459 - val_loss: 0.0813 - val_accuracy: 0.9734
Epoch 2/5
131/131 [==============================] - 19s 143ms/step - loss: 0.0200 - accuracy: 0.9950 - val_loss: 0.0507 - val_accuracy: 0.9849
Epoch 3/5
131/131 [==============================] - 19s 144ms/step - loss: 0.0150 - accuracy: 0.9962 - val_loss: 0.0337 - val_accuracy: 0.9878
Epoch 4/5
131/131 [==============================] - 19s 146ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.0501 - val_accuracy: 0.9849
Epoch 5/5
131/131 [==============================] - 19s 142ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0546 - val_accuracy: 0.9878


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  predicted = model.predict([pred_text])
  prediction = []
  if predicted[0][0] < 0.5:
    prediction.extend([predicted[0][0], 'ham'])
  else:
    prediction.extend([predicted[0][0], 'spam'])
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 2s 2s/step
[9.937384e-05, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 32ms/step
You passed the challenge. Great job!
